In [1]:
!pip install transformers accelerate datasets torch torchvision peft pillow

In [31]:
from datasets import (
load_dataset,
DatasetDict
)
from transformers import (
AutoTokenizer,
AutoModelForCausalLM,
TrainingArguments,
Trainer
)
from peft import (
LoraConfig,
get_peft_model,
TaskType,
PeftConfig,
PeftModel
)
import torch
from huggingface_hub import notebook_login

In [3]:
ds = load_dataset("aictsharif/persian-med-qa")

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 209384
    })
})

In [5]:
ds_small = DatasetDict({
    "train": ds["train"].select(range(1000))
})

In [6]:
ds_small

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1000
    })
})

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")

def preprocess(sample):
    sample = sample["question"] + "\n" + sample["answer"]
    tokenized = tokenizer(
        sample,
        max_length = 100,
        truncation = True,
        padding = "max_length"
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

In [8]:
data = ds_small.map(preprocess)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
data

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [10]:
print(data["train"][0])

{'question': 'علت سرماخوردگی چیست؟', 'answer': 'سرماخوردگی معمولاً به دلیل ویروس\u200cهای مختلفی مانند rhinovirus ایجاد می\u200cشود.', 'input_ids': [123987, 14293, 59842, 11071, 124009, 35244, 131089, 63732, 14391, 220, 144751, 14391, 46586, 128332, 198, 124537, 124009, 35244, 131089, 63732, 14391, 23364, 124423, 126897, 124376, 81768, 44330, 8532, 14391, 8532, 37524, 14391, 129236, 89364, 16157, 46072, 127519, 14391, 23364, 39423, 124523, 21669, 258, 859, 16972, 12961, 14391, 142928, 23364, 14391, 89364, 32790, 69423, 13, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    device_map = "cuda",
    torch_dtype = torch.float16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
lora_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    target_modules = ["q_proj", "k_proj", "v_proj"]
)

In [13]:
model = get_peft_model(model, lora_config)

In [14]:
training_args = TrainingArguments(
    num_train_epochs = 10,
    learning_rate = 0.001,
    logging_steps = 10,
    report_to = "tensorboard"
)

In [15]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = data["train"]
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [16]:
print("Start training...")
trainer.train()
print("Training finished...")

Start training...


Step,Training Loss
10,2.433900
20,0.867200
30,0.703700
40,0.579800
50,0.585000
60,0.548900
70,0.553900
80,0.561600
90,0.562500
100,0.482400


Training finished...


In [17]:
trainer.save_model("/kaggle/working/")
tokenizer.save_pretrained("/kaggle/working/")

('/kaggle/working/tokenizer_config.json',
 '/kaggle/working/special_tokens_map.json',
 '/kaggle/working/chat_template.jinja',
 '/kaggle/working/vocab.json',
 '/kaggle/working/merges.txt',
 '/kaggle/working/added_tokens.json',
 '/kaggle/working/tokenizer.json')

In [18]:
path = "/kaggle/working/"

In [19]:
config = PeftConfig.from_pretrained(path)
base = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code = True)
model = PeftModel.from_pretrained(base, path)
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code = True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
def generate_response(query):
    inputs = tokenizer(query, return_tensors = "pt").to(model.device)
    output = model.generate(
        input_ids = inputs["input_ids"],
        attention_mask = inputs["attention_mask"],
        max_new_tokens = 100
    )
    return tokenizer.decode(output[0])

In [21]:
print(generate_response("علائم آنفولانزا چیست؟"))

علائم آنفولانزا چیست؟
علائم آنفولانزا شامل تب، سرفه، گلودرد، بدن‌درد و خستگی است.<|endoftext|>


In [22]:
print(generate_response("چگونه می‌توان از بیماری دیابت جلوگیری کرد؟"))

چگونه می‌توان از بیماری دیابت جلوگیری کرد؟
با حفظ وزن سالم، ورزش منظم، و رژیم غذایی متعادل می‌توان از دیابت جلوگیری کرد.<|endoftext|>


In [23]:
print(generate_response("چگونه فشار خون را کنترل کنیم؟"))

چگونه فشار خون را کنترل کنیم؟
با تغییر رژیم غذایی، کاهش نمک، ورزش منظم و مصرف داروهای تجویز شده می‌توان فشار خون را کنترل کرد.<|endoftext|>


In [24]:
print(generate_response("علائم آلرژی چیست؟"))

علائم آلرژی چیست؟
علائم آلرژی شامل عطسه، خارش چشم، آبریزش بینی و کهیر است.<|endoftext|>


In [25]:
def generate_response2(query):
    inputs = tokenizer(query, return_tensors = "pt").to(model.device)
    output = model.generate(
        input_ids = inputs["input_ids"],
        attention_mask = inputs["attention_mask"],
        max_new_tokens = 100,
        do_sample = True,
        temperature = 0.7,
        top_k = 50,
        top_p = 0.9,
        repetition_penalty = 1.2
    )
    return tokenizer.decode(output[0], skip_special_tokens = True)

In [26]:
print(generate_response2("علائم آنفولانزا چیست؟"))

علائم آنفولانزا چیست؟
علائم آنفولانزا شامل تب، سرفه، گلو درد، بدن دردهای عضلانی و خستگی است.


In [27]:
print(generate_response2("چگونه می‌توان از بیماری دیابت جلوگیری کرد؟"))

چگونه می‌توان از بیماری دیابت جلوگیری کرد؟
با حفظ وزن سالم، ورزش منظم، و رژیم غذایی متعادل می‌توان از دیابت جلوگیری کرد.


In [28]:
print(generate_response2("چگونه فشار خون را کنترل کنیم؟"))

چگونه فشار خون را کنترل کنیم؟
با تغذیه سالم، ورزش منظم، استفاده از داروهای مصرف نیاز به کنترل فشار خون است.


In [29]:
print(generate_response2("علائم آلرژی چیست؟"))

علائم آلرژی چیست؟
علائم شامل عطسه، خارش و آبریزش بینی است.


In [33]:
notebook_login()

In [34]:
model.push_to_hub("alikhademi98/finetuned_Qwen2.5_on_persian_medical_qa")
tokenizer.push_to_hub("alikhademi98/finetuned_Qwen2.5_on_persian_medical_qa")

Uploading...:   0%|          | 0.00/10.1M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Uploading...:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alikhademi98/finetuned_Qwen2.5_on_persian_medical_qa/commit/916f7aba49847cf1b61af366de111a82079138cf', commit_message='Upload tokenizer', commit_description='', oid='916f7aba49847cf1b61af366de111a82079138cf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/alikhademi98/finetuned_Qwen2.5_on_persian_medical_qa', endpoint='https://huggingface.co', repo_type='model', repo_id='alikhademi98/finetuned_Qwen2.5_on_persian_medical_qa'), pr_revision=None, pr_num=None)